# Limpiar barcodes

## Parámetros

In [2]:
BASE_DIR = '/Users/efraflores/Downloads'
FILE_NAME = 'Barcodes Catalog MX.csv'

## Código

In [4]:
from numpy import nan
from pathlib import Path
from re import sub, UNICODE
from unicodedata import normalize
from pandas import ExcelFile, read_csv

BASE_DIR = Path(BASE_DIR)

def clean_text(text: str, pattern: str="[^a-zA-Z0-9\s,]", lower: bool=False) -> str: 
    '''
    Limpieza de texto
    '''
    # Reemplazar acentos: áàäâã --> a
    clean = normalize('NFD', str(text).replace('\n', ' \n ')).encode('ascii', 'ignore')
    # Omitir caracteres especiales !"#$%&/()=...
    clean = sub(pattern, ' ', clean.decode('utf-8'), flags=UNICODE)
    # Mantener sólo un espacio
    clean = sub(r'\s{2,}', ' ', clean.strip())
    # Minúsculas si el parámetro lo indica
    if lower: clean = clean.lower()
    # Si el registro estaba vacío, indicar nulo
    if clean in ('','nan'): clean = nan
    return clean

## Importar

In [5]:
data = read_csv(BASE_DIR.joinpath(FILE_NAME)).astype(str)
# data = ExcelFile(BASE_DIR.joinpath(FILE_NAME)).parse(sheet_name=0).astype(str)
data.sample()

,Currency,Product ID,Name,Barcodes
165198,MXN,566535,Lápices pastel conté soft estuche con 20,{646217502083}


## Transformar

### Limpiar texto y separar por columnas

In [6]:
barcodes = data['Barcodes'].map(lambda x: clean_text(x, pattern=r'[\{\}\"\']')).str.split(',', expand=True).set_index(data.index)
barcodes.columns = [f'barcodes_{col+1}' for col in barcodes.columns]
barcodes.sample()

,barcodes_1,barcodes_2,barcodes_3,barcodes_4,barcodes_5,barcodes_6,barcodes_7,barcodes_8,barcodes_9,barcodes_10
376287,8020312002709,None,None,None,None,None,None,None,None,None


### Unir con tabla original

In [7]:
df = data.join(barcodes)
del data, barcodes
df[df['barcodes_7'].notnull()].sample()

,Currency,Product ID,Name,Barcodes,barcodes_1,barcodes_2,barcodes_3,barcodes_4,barcodes_5,barcodes_6,barcodes_7,barcodes_8,barcodes_9,barcodes_10
132276,MXN,180724,Capeador para pollo Shake 'n Bake,"{43079032,9999917012750,4307903,43000057889,70...",43079032,9999917012750,4307903,43000057889,7052830430796,7043000000796,43000000793,None,None,None


### Sólo una columna para barcodes

In [8]:
df = df.melt(id_vars=['Currency', 'Product ID', 'Name', 'Barcodes'], var_name='barcode_n', value_name='barcode').dropna()
df.sample()

,Currency,Product ID,Name,Barcodes,barcode_n,barcode
335317,MXN,3269192,Olmesartán medoxomilo tabletas 40 mg,{7506442700773},barcodes_1,7506442700773


### Agrupar product ID para cada barcode

In [9]:
df = df.pivot_table(index='barcode', values='Product ID', aggfunc=', '.join)
df.sample()

,Product ID
barcode,
7502009747144,2389181


### Expandir al número de product_ID que tenga cada barcode

In [10]:
df = df['Product ID'].astype(str).str.split(',', expand=True)
df.columns = [f'product_id_{col+1}' for col in df.columns]
df.sample()

,product_id_1,product_id_2,product_id_3,product_id_4,product_id_5,product_id_6,product_id_7,product_id_8,product_id_9,product_id_10,product_id_11,product_id_12
barcode,,,,,,,,,,,,
7501244003268,579599,None,None,None,None,None,None,None,None,None,None,None


### Ordenar los barcodes con más product_IDs

In [11]:
df.sort_values(list(reversed(df.columns)), inplace=True)
df.reset_index(inplace=True)
df.head(3)

,barcode,product_id_1,product_id_2,product_id_3,product_id_4,product_id_5,product_id_6,product_id_7,product_id_8,product_id_9,product_id_10,product_id_11,product_id_12
0,7501055361199,4599926,4594342,4718214,4720588,4599927,4638703,4601649,4601215,4528681,4601212,4597388,336093
1,7501055368440,4601171,4638683,1256963,4718142,4597367,4601174,4601554,4599885,4599887,4720571,336764,None
2,7501055368464,4720567,4638679,4601168,4601521,4599876,4597364,1256965,4599879,4601163,4718138,337428,None


### Agregar ' al barcode para que Excel no omita los 0 a la izquierda

In [12]:
df['barcode'] = df['barcode'].map(lambda x: "'"+x)
df.sample()

,barcode,product_id_1,product_id_2,product_id_3,product_id_4,product_id_5,product_id_6,product_id_7,product_id_8,product_id_9,product_id_10,product_id_11,product_id_12
202873,'7500539079599,2443661,None,None,None,None,None,None,None,None,None,None,None


### Exportar resultado

In [13]:
new_name = ''.join(FILE_NAME.split('.')[:-1])
df.to_csv(BASE_DIR.joinpath(f'clean_{new_name}.csv'), sep='\t', encoding='utf-16', index=False)